<a href="https://colab.research.google.com/github/CharlottePrimiceri/VP_Project/blob/main/depthmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install timm
import cv2
import os
import numpy as np
import torch
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.7 MB/s eta 0:00:00


In [5]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
model_type = "DPT_Hybrid"
midas = torch.hub.load("intel-isl/MiDaS", model_type)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
  transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

In [ ]:
destination_folder = "/content/drive/MyDrive/VPPROJECT/cityscapes/depth_images/test"
#imgs = os.listdir(folder_dir)
for city in os.listdir(folder_dir):
    img_dir = os.path.join(folder_dir, city)
    new_dir = os.path.join(destination_folder, city)
    #new_dir = os.path.join(destination_folder, os.path.relpath(os.path.join(root, directory), input_folder))
    os.makedirs(new_dir, exist_ok=True)
    for imgid in os.listdir(img_dir):
        path = os.path.join(img_dir,imgid)

        output_path = os.path.join(destination_folder, os.path.relpath(path, folder_dir))
        output_dir = os.path.dirname(output_path)
        os.makedirs(output_dir, exist_ok=True)

        img = cv2.imread(path)
        input_batch = transform(img).to(device)
        with torch.no_grad():
             prediction = midas(input_batch)
             prediction = torch.nn.functional.interpolate(
             prediction.unsqueeze(1),
             size=img.shape[:2],
             mode="bicubic",
             align_corners=False,
             ).squeeze()
        depth = prediction.cpu().numpy()
        normalized_depth = (depth - np.min(depth)) / (np.max(depth) - np.min(depth))
        depth_rgb = cv2.applyColorMap((normalized_depth * 255).astype(np.uint8), cv2.COLORMAP_JET)
        cv2.imwrite(output_path, depth_rgb)